In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
from kan import KAN

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

from enum import Enum

In [ ]:
data = fetch_california_housing()
column_names = data['feature_names']
X = data['data']
y = data['target']

print(column_names) # y = 'MedInc'
print(X.shape, y.shape)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
(20640, 8) (20640,)


In [100]:
class DSubset(Enum):
    TRAIN = 1
    TEST = 2

class CaliforniaHousingDataset(Dataset):
    """
    Represents the California Housing Price dataset from scikit-learn.
    """
    def __init__(self, X: np.ndarray, y: np.ndarray, split: DSubset, train_size = 0.8, transform = None, target_transform = None, standard_scaler: StandardScaler = None):
        """
        Initializes dataset, properly scaling X for training.
        """
        num_train = int(train_size * y.shape[0])

        if split == DSubset.TRAIN:
            self.X = X[:num_train]
            self.y = y[:num_train]
        else:
            self.X = X[num_train:]
            self.y = y[num_train:]

        if split == DSubset.TRAIN:
            assert not standard_scaler
            self.ss = StandardScaler()
            self.X = self.ss.fit_transform(self.X)
        else:
            assert standard_scaler
            self.X = standard_scaler.transform(self.X)

        self.X = torch.tensor(self.X).float()
        self.y = torch.tensor(self.y).float()

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        """
        Provides the length of the dataset.
        """
        return len(self.y)

    def __getitem__(self, idx):
        """
        Provides the instance and label at the corresponding index within
        the dataset.
        """
        instance, label = self.X[idx], self.y[idx]

        if self.transform:
            instance = self.transform(instance)
        
        if self.target_transform:
            label = self.target_transform(label)

        return instance, label

    def get_standard_scaler(self) -> StandardScaler:
        return self.ss

In [106]:
TRAIN_SIZE = 0.8

train_dataset = CaliforniaHousingDataset(X, y, DSubset.TRAIN, train_size = TRAIN_SIZE)
test_dataset = CaliforniaHousingDataset(X, y, DSubset.TEST, train_size = TRAIN_SIZE, standard_scaler = train_dataset.get_standard_scaler())

BATCH_SIZE = 32

train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [102]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn, optimizer: torch.optim.Optimizer):
    """
    Runs one full epoch of training on model.

    Args:
        dataloader -- The DataLoader through which to produce instances.
        model -- The model to be used for label prediction on instances.
        loss_fn -- The loss function, for backpropagation
        optimizer -- The optimizer, for reducing loss

    Returns:
        average_epoch_loss -- The model's loss this epoch, averaged by the number of instances in the dataset
    """
    model.train()

    epoch_loss = 0.0

    for i, (X, y) in enumerate(dataloader):

        # Forward
        pred = model(X)
        batch_loss = loss_fn(pred.squeeze(1), y)

        # Log
        epoch_loss += batch_loss.item()

        # Backpropagate
        batch_loss.backward()

        # Optimize
        optimizer.step()
        optimizer.zero_grad()

        # Display
        # print(f'Batch {i + 1} Loss: {batch_loss}')

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

def run_test_epoch(dataloader: DataLoader, model: nn.Module, loss_fn):
    """
    Runs one full dataset-worth of testing on model.

    Args:
        dataloader -- The DataLoader through which to produce instances.
        model -- The model to be used for label prediction on instances.
        loss_fn -- The loss function, for measuring generalizability

    Returns:
        average_epoch_loss -- The model's loss this epoch, averaged by the number of instances in the dataset
    """
    model.eval()

    epoch_loss = 0.0

    with torch.no_grad():

        for X, y in dataloader:

            # Forward
            pred = model(X)
            batch_loss = loss_fn(pred.squeeze(1), y)

            # Log
            epoch_loss += batch_loss.item()

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

In [124]:
model = KAN(len(column_names), 1)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [104]:
EPOCHS = 10

for i in range(EPOCHS):
    avg_train_loss = run_train_epoch(train_dataloader, model, loss_fn, optimizer)
    avg_test_loss = run_test_epoch(test_dataloader, model, loss_fn)

    print(f'Epoch {i + 1:>3} | Train Loss: {avg_train_loss:>9.4f} | Test Loss: {avg_test_loss:>9.4f}')

Epoch   1 | Train Loss:    0.0279 | Test Loss:    0.0212
Epoch   2 | Train Loss:    0.0176 | Test Loss:    0.0177
Epoch   3 | Train Loss:    0.0151 | Test Loss:    0.0170
Epoch   4 | Train Loss:    0.0141 | Test Loss:    0.0166
Epoch   5 | Train Loss:    0.0130 | Test Loss:    0.0173
Epoch   6 | Train Loss:    0.0121 | Test Loss:    0.0143
Epoch   7 | Train Loss:    0.0117 | Test Loss:    0.0171
Epoch   8 | Train Loss:    0.0114 | Test Loss:    0.0133
Epoch   9 | Train Loss:    0.0112 | Test Loss:    0.0146
Epoch  10 | Train Loss:    0.0110 | Test Loss:    0.0145
